# ML: Breast Cancer

## Strategy

- Data split:
    - Data is split into **train (90%)** and **test (10%)**
    - Train data is used for cross-validation as well when performing hyperparameter tuning 
- Null/empty values:
    - Used **SimpleImputer**:
        - numerical columns: **mean**
        - categorical columns: **most_frequent**
- Encoding:
    - y data: **LabelEncoder**
    - x data: **TargetEncoder**
- Feature selection:
    - Used **RandomForestClassifier** and **SelectFromModel** to select the needed features 
    - Removed features from x_train data
- Model training:
    - Used **XGBClassifier** as it was stated in multiple papers for yielding the best results 
    - Used **f1_macro** for scoring 
    - Fit using **train** data
- Testing:
    - Pre-processing done:
        - Handle null/empty values similar to the handling of the train data
        - Encode the test data 
        - Remove feature from **x_test** similar to the ones removed from the **x_train**
    - Generate predictions
    - Display the following metrics:
        - Accuracy 
        - Precision 
        - Recall 
        - F1 Score 

## Imports

In [1]:
!pip install category_encoders

In [4]:
import numpy as np
import pandas as pd
import sklearn
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

## Save/Get Model

In [5]:
path = "C:\\Users\\aidaf\\OneDrive\\Documents\\University\\Spring 2023-2024\\CMPS 396V Machine Learning\\Project\\ML_Breast_Cancer\\Aida\\"
# path = "D:\\Desktop\\ML\\Project\\ML_Breast_Cancer\\Models\\"

def saveModel(fileName, model):
  with open(path + fileName, 'wb') as f:
      pickle.dump(model, f)

def getModel(fileName):
  with open(path + fileName, 'rb') as f:
      loaded_model = pickle.load(f)
  return loaded_model

## Load and Split the Data

In [6]:
data = pd.read_csv("data.csv", encoding="utf-8")

y = data["cancer_type"]
x = data.drop(columns = ["cancer_type", "patient_id"])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True, random_state=3)

C:\Users\aidaf\AppData\Local\Temp\ipykernel_1732\2955086971.py:1: DtypeWarning: Columns (662,664,676,677,683,685,686,687) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data.csv", encoding="utf-8")


## Training

### Fill Null/Empty Values

In [7]:
def get_categorical_columns(df):
    return list(df.select_dtypes(include=['object', 'category']).columns)
def get_numerical_columns(df):
    return list(df.select_dtypes(include=['number']).columns)

In [8]:
categorical_columns = get_categorical_columns(x_train)
numerical_columns = get_numerical_columns(x_train)

numerical_imputer = SimpleImputer(strategy='mean') 
x_train[numerical_columns] = numerical_imputer.fit_transform(x_train[numerical_columns])

categorical_imputer = SimpleImputer(strategy='most_frequent')  
x_train[categorical_columns] = categorical_imputer.fit_transform(x_train[categorical_columns])

x_train.shape

(1346, 686)

### Encode x_train and y_train

In [9]:
y_train = LabelEncoder().fit_transform(y_train)

encoder = ce.TargetEncoder(cols=categorical_columns)
x_train = encoder.fit_transform(x_train, y_train)

### Apply Feature Selection 

In [10]:
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(rf_classifier)),
    ('classification', rf_classifier)
])

param_grid = {'feature_selection__threshold': ['mean', 'median', '1.25*mean']}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

best_pipeline = grid_search.best_estimator_

print("Selected Features Threshold:", best_pipeline.named_steps['feature_selection'].threshold)

selected_features_mask = best_pipeline.named_steps['feature_selection'].get_support()
x_train = x_train.iloc[:, selected_features_mask]

Selected Features Threshold: 1.25*mean


### XGBoost Model Training

In [11]:
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200, 300],
}

xgb_model = XGBClassifier()
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_score = grid_search.best_score_
print("Best Cross-Validation Score:", best_score)

best_model = grid_search.best_estimator_
print("Best Model:", best_model)

saveModel('best_xgb_model.pkl', best_model)

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 100}
Best Cross-Validation Score: 0.6179617328464837
Best Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)


## Testing

### Handle Null/Empty Values of Test Data

In [12]:
categorical_columns = get_categorical_columns(x_test)
numerical_columns = get_numerical_columns(x_test)

numerical_imputer = SimpleImputer(strategy='mean') 
x_test[numerical_columns] = numerical_imputer.fit_transform(x_test[numerical_columns])

categorical_imputer = SimpleImputer(strategy='most_frequent') 
x_test[categorical_columns] = categorical_imputer.fit_transform(x_test[categorical_columns])

### Apply Encoding to Test Data

In [13]:
y_test = LabelEncoder().fit_transform(y_test)

encoder = ce.TargetEncoder(cols=categorical_columns)
x_test = encoder.fit_transform(x_test, y_test)

### Apply Feature Selection on Test Data

In [14]:
x_test = x_test.iloc[:, selected_features_mask]

### Test and Display Metrics 

In [15]:
test_pred = best_model.predict(x_test)

accuracy = accuracy_score(y_test, test_pred)
precision = precision_score(y_test, test_pred, average="macro")
recall = recall_score(y_test, test_pred, average="macro")
f1 = f1_score(y_test, test_pred, average="macro")

print("accuracy =", accuracy)
print("precision =", precision)
print("recall =", recall)
print("f1-score =", f1)

accuracy = 0.8666666666666667
precision = 0.8140497967479675
recall = 0.630031779661017
f1-score = 0.6908500904351527
